In [1]:
# https://goo.gl/6TWchp
import os
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import pydicom
from imgaug import augmenters as iaa

import skimage.io
import skimage.measure
from tqdm import tqdm
from PIL import Image

import requests
import shutil
import zipfile

In [2]:
import mdai
mdai.__version__

'0.0.5'

In [3]:
mdai_client = mdai.Client(domain='public.md.ai', access_token=os.environ['MDAI_TOKEN'])

Successfully authenticated to public.md.ai.


In [8]:
mdai_client.

{'domain': 'public.md.ai',
 'access_token': 'cab187e530482790d5ac6f9769aa260d',
 'session': <requests.sessions.Session at 0x11f9fbda0>}

In [ ]:
p = mdai_client.project('aGq4k6NW', path='../data/raw/lesson2-data', force_download=True)

Using path '../data/raw/lesson2-data' for data.
Preparing annotations export for project aGq4k6NW...                                                
Preparing images export for project aGq4k6NW...                                                     


3.75MB [00:01, 3.12MB/s]                            


Success: annotations data for project aGq4k6NW ready.


Exception in thread Thread-15041:
Traceback (most recent call last):
  File "/Users/ylchen/.virtualenvs/kaggle-rsna/lib/python3.6/site-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/Users/ylchen/.virtualenvs/kaggle-rsna/lib/python3.6/site-packages/urllib3/connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/Users/ylchen/.virtualenvs/kaggle-rsna/lib/python3.6/site-packages/urllib3/connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/ylchen/.pyenv/versions/3.6.4/lib/python3.6/http/client.py", line 1331, in getresponse
    response.begin()
  File "/Users/ylchen/.pyenv/versions/3.6.4/lib/python3.6/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/Users/ylchen/.pyenv/versions/3.6.4/lib/python3.6/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "

In [5]:
# download MD.ai's dilated unet implementation 
UNET_URL = 'https://s3.amazonaws.com/md.ai-ml-lessons/unet.zip'
UNET_ZIPPED = '../data/raw/unet.zip'

if not os.path.exists(UNET_ZIPPED): 
    r = requests.get(UNET_URL, stream=True)
    if r.status_code == requests.codes.ok:
        with open(UNET_ZIPPED, "wb") as f:
            shutil.copyfileobj(r.raw, f)
    else:
        r.raise_for_status()

    with zipfile.ZipFile(UNET_ZIPPED) as zf:
        zf.extractall()

In [6]:
p.show_label_groups()

NameError: name 'p' is not defined

In [ ]:
# this maps label ids to class ids as a dict obj
labels_dict = {'L_A8Jm3d':1 # Lung   
              }

print(labels_dict)
p.set_labels_dict(labels_dict)

In [ ]:
p.show_datasets()

In [ ]:
dataset = p.get_dataset_by_id('D_rQLwzo')
dataset.prepare()

In [ ]:
# visualize a few train images 
mdai.visualize.display_images(image_ids[:3], cols=2)

In [ ]:
imgs_anns_dict = dataset.imgs_anns_dict

In [ ]:
from unet import dataset
from unet import dilated_unet
from unet import train

In [ ]:
images, masks = dataset.load_images(imgs_anns_dict)

In [ ]:
img_index = random.choice(range(len(imgs_anns_dict)))

print(img_index)
img_fps = list(imgs_anns_dict.keys())
img_fp = img_fps[img_index]
img = mdai.visualize.load_dicom_image(img_fp)
ann = imgs_anns_dict[img_fp]
img_width = img.shape[1]
img_height = img.shape[0]

mask = np.zeros((img_height, img_width), dtype=np.uint8) 
for a in ann:     
    vertices = np.array(a['data']['vertices'])
    vertices = vertices.reshape((-1,2))                     
    cv2.fillPoly(mask, np.int32([vertices]), (255,255,255))
    
plt.figure(figsize=(30, 20))
plt.subplot(2,3,1)
plt.imshow(img, cmap=plt.cm.bone)
plt.axis('off')

plt.subplot(2,3,2)
plt.imshow(mask, cmap=plt.cm.bone)
plt.axis('off')

plt.subplot(2,3,3)              
plt.imshow(cv2.bitwise_and(img, img, mask=mask.astype(np.uint8)), cmap=plt.cm.bone)
plt.axis('off')

plt.subplot(2,3,4)
plt.imshow(images[img_index,:,:,0], cmap=plt.cm.bone)
plt.axis('off')

plt.subplot(2,3,5)
plt.imshow(masks[img_index,:,:,0], cmap=plt.cm.bone)
plt.axis('off')

plt.subplot(2,3,6)
plt.imshow(cv2.bitwise_and(images[img_index,:,:,0], images[img_index,:,:,0], 
                           mask=masks[img_index,:,:,0].astype(np.uint8)), cmap=plt.cm.bone)
plt.axis('off')

In [ ]:
import tensorflow as tf 
config = tf.ConfigProto()
config.gpu_options.allow_growth = True